# DFCI analysis

In [ ]:
import numpy as np
import pandas as pd
import os
import torch
from functools import partial

import contextlib
import io
import warnings
import json

from madrigal.utils import BASE_DIR
from madrigal.evaluate.predict import get_twosides_scores_wrapper

from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
from scipy.stats import kendalltau

def sigmoid(x: np.ndarray):
    return 1 / (1 + np.exp(-x))

drug_metadata = pd.read_pickle(os.path.join(BASE_DIR, 'processed_data/views_features_new/combined_metadata_ddi.pkl'))
drug_metadata['view_str'] = 1
print(drug_metadata.shape[0])

11601
21842


In [ ]:
twosides_ddi_classes = pd.read_pickle(
    BASE_DIR + "processed_data/polypharmacy_new/TWOSIDES/twosides_ddi_directed_final_label_map.pkl"
)
twosides_ckpts = ['effortless-dust-7', 'fresh-flower-8', 'vague-violet-9']
get_twosides_scores = partial(get_twosides_scores_wrapper, twosides_ddi_classes=twosides_ddi_classes, ckpt_list=twosides_ckpts)

outcome_mapper = json.load(open("../outcome_mapper.json", "r"))
outcome_inds_mapper = {
    ae_dfci: {"twosides": [twosides_ddi_classes.tolist().index(ae_db) for ae_db in dct["twosides"]]} 
    for ae_dfci, dct in outcome_mapper.items()
}

## Data preparation

Due to patient privacy concerns, patient-level data is not released.

In [6]:
dfci_patient_data = pd.read_csv("./dfci/patient_level_ae_dfci_relaxed.csv", index_col=0)
dfci_patient_data.columns

Index(['FIRST_DRUG_REGIMEN', 'ICD_BASED_TISSUE_TYPE', 'PALLIATIVE_INTENT',
       'RACE', 'GENDER_NM', 'AGE_AT_TREAT', 'AE_neutropenia',
       'AE_pancytopenia', 'AE_anemia', 'AE_thrombocytopenia',
       'AE_polyneuropathy', 'AE_pulmonary_embolism/deep_vein_thrombosis',
       'AE_acute_kidney_injury', 'AE_hyponatremia', 'AE_hypokalemia',
       'AE_hyperkalemia', 'AE_hypomagnesemia', 'AE_hypocalcemia',
       'AE_hypercalcemia'],
      dtype='object')

In [ ]:
temp = dfci_patient_data["FIRST_DRUG_REGIMEN"].value_counts()
temp = temp[temp.index.str.contains("\+")]
assert temp[temp.index.str.contains("SALINE")].shape[0] == 0
assert temp[(temp.index.str.contains("/")) & (temp >= 10)].shape[0] == 0

dfci_patient_data_filtered = dfci_patient_data[dfci_patient_data["FIRST_DRUG_REGIMEN"].str.contains("\+")]
dfci_patient_data_filtered[["drug_name_1", "drug_name_2"]] = dfci_patient_data_filtered["FIRST_DRUG_REGIMEN"].str.replace(" HYDROCHLORIDE", "").str.replace(" MALEATE", "").str.replace(" MESYLATE", "").str.replace(" CITRATE", "").str.replace(" TRIOXIDE", "").str.replace(" TARTRATE", "").str.replace(" EMTANSINE", "").str.replace("PEGYLATEDLIPOSOMALDOXORUBICIN", "DOXORUBICIN").str.replace(" TOSYLATE", "").str.split("\+", expand=True)
dfci_patient_data_filtered[["drug_name_1", "drug_name_2"]] = dfci_patient_data_filtered[["drug_name_1", "drug_name_2"]].apply(lambda row: [row["drug_name_1"].strip().capitalize(), row["drug_name_2"].strip().capitalize()], axis=1, result_type="expand")
dfci_patient_data_filtered["drug_name_1"] = dfci_patient_data_filtered["drug_name_1"].str.replace("Mesna", "Coenzyme M").str.replace("Arsenic", "Arsenic trioxide").str.replace("Mirdametinib", "PD-0325901")
dfci_patient_data_filtered["drug_name_2"] = dfci_patient_data_filtered["drug_name_2"].str.replace("Mesna", "Coenzyme M").str.replace("Arsenic", "Arsenic trioxide").str.replace("Mirdametinib", "PD-0325901")
dfci_patient_data_filtered = dfci_patient_data_filtered[~(dfci_patient_data_filtered["drug_name_1"].str.endswith("mab") | dfci_patient_data_filtered["drug_name_2"].str.endswith("mab") | dfci_patient_data_filtered["drug_name_1"].str.endswith("vedotin") | dfci_patient_data_filtered["drug_name_2"].str.endswith("vedotin") | dfci_patient_data_filtered["drug_name_1"].isin({"Sargramostim", "Tagraxofusp-erzs", "Wee1 inhibitor zn-c3", "Sacituzumab govitecan", "Azenosertib"}) | dfci_patient_data_filtered["drug_name_2"].isin({"Sargramostim", "Tagraxofusp-erzs", "Wee1 inhibitor zn-c3", "Sacituzumab govitecan", "Azenosertib"}))]  # remove monoclonal antibodies or drugs not in our database
dfci_patient_data_filtered = dfci_patient_data_filtered[dfci_patient_data_filtered["FIRST_DRUG_REGIMEN"].isin(dfci_patient_data_filtered["FIRST_DRUG_REGIMEN"].value_counts()[dfci_patient_data_filtered["FIRST_DRUG_REGIMEN"].value_counts() >= 10].index.values)]

dfci_patient_data_filtered[["dbid_1", "dbid_2"]] = dfci_patient_data_filtered[["drug_name_1", "drug_name_2"]].apply(lambda row: [drug_metadata[drug_metadata["node_name"] == row["drug_name_1"]]["node_id"].values[0], drug_metadata[drug_metadata["node_name"] == row["drug_name_2"]]["node_id"].values[0]], axis=1, result_type="expand")
dfci_patient_data_filtered[["drug_index_1", "drug_index_2"]] = dfci_patient_data_filtered[["dbid_1", "dbid_2"]].apply(lambda row: [drug_metadata[drug_metadata["node_id"] == row["dbid_1"]].index.values[0], drug_metadata[drug_metadata["node_id"] == row["dbid_2"]].index.values[0]], axis=1, result_type="expand")

dfci_patient_data_filtered.to_pickle("./dfci/dfci_patient_data_filtered.pkl")

In [ ]:
aes = [col[len("AE_"):] for col in dfci_patient_data_filtered.columns if col.startswith("AE_")]
# aes = ["neutropenia", "pancytopenia", "anemia", "thrombocytopenia", "polyneuropathy", "pulmonary_embolism/deep_vein_thrombosis", "acute_kidney_injury", "hyponatremia", "hypokalemia", "hyperkalemia", "hypomagnesemia", "hypocalcemia", "hypercalcemia"]

# First generate scores for each drug for each outcome
drug_names = np.unique(dfci_patient_data_filtered[["drug_name_1", "drug_name_2"]].values.flatten())
drug_inds = [drug_metadata[drug_metadata["node_name"] == drug_name].index.values[0] for drug_name in drug_names]

# Then generate scores for each drug pair for each outcome
drug_1_ind_inds = [drug_names.tolist().index(name) for name in dfci_patient_data_filtered["drug_name_1"].tolist()]
drug_2_ind_inds = [drug_names.tolist().index(name) for name in dfci_patient_data_filtered["drug_name_2"].tolist()]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    with contextlib.redirect_stdout(io.StringIO()):
        dfci_patient_drugs_twosides_scores, _ = get_twosides_scores(
            outcome_twosides_inds=sum([[o for o in outcome_inds_mapper[ae]["twosides"] if o not in {twosides_ddi_classes.tolist().index("Adverse event"), twosides_ddi_classes.tolist().index("Adverse drug reaction")}] for ae in aes], start=[]), 
            drug_inds=drug_inds, 
            drug_group_str="dfci_patient",
        )
        
pd.DataFrame(dfci_patient_drugs_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([[o for o in outcome_mapper[ae]["twosides"] if o not in {"Adverse event", "Adverse drug reaction"}] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).to_pickle(BASE_DIR + "temp/dfci_patient_drugs_twosides_scores.pkl")

## Compute correlation

Map data

In [ ]:
dfci_data = pd.read_pickle("./dfci/dfci_patient_data_filtered.pkl")
temp = dfci_data.groupby("FIRST_DRUG_REGIMEN").agg(list)
dfci_data = dfci_data[dfci_data["FIRST_DRUG_REGIMEN"].isin(temp[temp["ICD_BASED_TISSUE_TYPE"].apply(len) >= 20].index.values)]
dfci_data = dfci_data.query("ICD_BASED_TISSUE_TYPE not in @heme_tissues")
dfci_data = dfci_data[["FIRST_DRUG_REGIMEN", "drug_name_1", "drug_name_2", "dbid_1", "dbid_2", "drug_index_1", "drug_index_2"] + [col for col in dfci_data.columns if col.startswith("AE_")]]
dfci_data = dfci_data[~dfci_data["FIRST_DRUG_REGIMEN"].str.contains("PEGYLATEDLIPOSOMALDOXORUBICIN")]  # no match
print(dfci_data.shape[0])

dfci_data = dfci_data.groupby("FIRST_DRUG_REGIMEN").agg(list)
dfci_data["drug_name_1"] = dfci_data["drug_name_1"].apply(lambda x: x[0])
dfci_data["drug_name_2"] = dfci_data["drug_name_2"].apply(lambda x: x[0])
dfci_data["dbid_1"] = dfci_data["dbid_1"].apply(lambda x: x[0])
dfci_data["dbid_2"] = dfci_data["dbid_2"].apply(lambda x: x[0])
dfci_data["drug_index_1"] = dfci_data["drug_index_1"].apply(lambda x: x[0])
dfci_data["drug_index_2"] = dfci_data["drug_index_2"].apply(lambda x: x[0])
dfci_data["num_all_patients"] = dfci_data["AE_neutropenia"].apply(lambda x: len(x))
for col in dfci_data.columns:
    if col.startswith("AE_"):
        ae = col[3:]
        dfci_data[f"num_{ae}_cases"] = dfci_data[col].apply(lambda lst: sum(lst))

dfci_data = pd.concat([
    dfci_data, 
    dfci_data.rename(
        columns={
            "drug_name_2": "drug_name_1", 
            "drug_name_1": "drug_name_2", 
            "dbid_2": "dbid_1", 
            "dbid_1": "dbid_2", 
            "drug_index_2": "drug_index_1", 
            "drug_index_1": "drug_index_2"
        }
    )
], axis=0)
dfci_data = dfci_data.query("drug_index_1 > drug_index_2").drop(columns=[col for col in dfci_data.columns if col.startswith("AE_")])
dfci_data = dfci_data.drop_duplicates(subset=["drug_index_1", "drug_index_2", "num_all_patients", "num_neutropenia_cases"])

In [10]:
drug_names = np.unique(dfci_data[["drug_name_1", "drug_name_2"]].values.flatten())
drug_inds = [drug_metadata[drug_metadata["node_name"] == drug_name].index.values[0] for drug_name in drug_names]
drug_1_ind_inds = [drug_names.tolist().index(name) for name in dfci_data["drug_name_1"].tolist()]
drug_2_ind_inds = [drug_names.tolist().index(name) for name in dfci_data["drug_name_2"].tolist()]

Set a detection threshold based on exact binomial test

In [11]:
import math
def n_detect(p=0.05, power=0.95):
    return math.ceil(math.log(1-power) / math.log(1-p))
n_detect(0.05, 0.8)

32

In [12]:
detect_thres = 32

In [ ]:
heme_drugs = {"Acalabrutinib", "Arsenic trioxide", "Azacitidine", "Bortezomib", "Busulfan", "Cytarabine", "Dasatinib", "Daunorubicin", "Decitabine", "Duvelisib", "Fludarabine", "Gilteritinib", "Ixazomib", "Lenalidomide", "Midostaurin", "Romidepsin", "Ruxolitinib", "Tretinoin", "Umbralisib", "Venetoclax", "Vincristine", "Imatinib", "Hydroxyurea", "Cyclophosphamide", "Methotrexate"}
heme_tissues = {"Myeloid", "Lymphoid", "Myeloma", "Leukemia", "Hematologic Other"}

Calculate scores

In [13]:
for ae in aes:
    dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
    dfci_data[["drug_name_1", "drug_name_2", "drug_index_1", "drug_index_2", f"proportion_{ae}"]].sort_values(f"proportion_{ae}", ascending=False)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        with contextlib.redirect_stdout(io.StringIO()):
            dfci_test_twosides_scores, _ = get_twosides_scores(
                outcome_twosides_inds=sum([outcome_inds_mapper[ae]["twosides"] for ae in aes], start=[]), 
                drug_inds=drug_inds, 
                drug_group_str="dfci_test",
            )
            
    torch.save(dfci_test_twosides_scores, BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae.replace('/', '_')}_scores.pt")

### Per AE correlation

#### neutropenia

In [ ]:
ae = "neutropenia"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")

dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Neutropenia"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(plot_data.shape)
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

(16, 34)
SignificanceResult(statistic=0.6614950926316517, pvalue=0.0004995129391708896)


#### pancytopenia

In [ ]:
ae = "pancytopenia"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")

dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Pancytopenia"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(plot_data.shape)
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

(16, 35)
SignificanceResult(statistic=0.41286141192238524, pvalue=0.03369680195497631)


#### anemia 

In [ ]:
ae = "anemia"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")

dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Anaemia"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

(16, 36)
SignificanceResult(statistic=0.39330888211518983, pvalue=0.03415760720580079)


#### thrombocytopenia

In [ ]:
ae = "thrombocytopenia"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")

dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Thrombocytopenia"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

(16, 37)
SignificanceResult(statistic=0.5222329678670935, pvalue=0.005992862170204803)


#### polyneuropathy

In [ ]:
ae = "polyneuropathy"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")

dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Neuropathy peripheral"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

SignificanceResult(statistic=0.4426266681379905, pvalue=0.023342202012890816)


#### pulmonary_embolism/deep_vein_thrombosis

In [ ]:
ae = "pulmonary_embolism/deep_vein_thrombosis"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae.replace('/', '_')}_scores.pt")

dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Deep vein thrombosis"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

(16, 39)
SignificanceResult(statistic=0.6102571532587294, pvalue=0.001125715058012237)


#### acute_kidney_injury

In [ ]:
ae = "acute_kidney_injury"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")
dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Nephropathy toxic"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

(16, 40)
SignificanceResult(statistic=0.3460192837535861, pvalue=0.06390794473460158)


#### hyponatremia

In [ ]:
ae = "hyponatremia"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")

dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Hyponatraemia"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

SignificanceResult(statistic=0.6839855680567694, pvalue=0.0002792104157155115)


#### hypokalemia

In [ ]:
ae = "hypokalemia"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")

dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Hypokalaemia"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

(16, 42)
SignificanceResult(statistic=0.3460192837535861, pvalue=0.06390794473460158)


#### hyperkalemia

In [ ]:
ae = "hyperkalemia"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")

dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Hyperkalaemia"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))


SignificanceResult(statistic=0.5378528742004771, pvalue=0.007028071105256244)


#### hypomagnesemia

In [ ]:
ae = "hypomagnesemia"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")
dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Hypomagnesaemia"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

SignificanceResult(statistic=0.5968834402710812, pvalue=0.0018896405048027735)


#### hypocalcemia

In [ ]:
ae = "hypocalcemia"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")
dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Hypocalcaemia"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

SignificanceResult(statistic=0.2604237178532571, pvalue=0.20432776343895498)


#### hypercalcemia

In [ ]:
ae = "hypercalcemia"
dfci_data[f"proportion_{ae}"] = dfci_data[f"num_{ae}_cases"] / dfci_data["num_all_patients"]
dfci_test_twosides_scores = torch.load(BASE_DIR + f"temp/final_drug_regimen_ae_summary_dfci_relaxed_v2_{ae}_scores.pt")

dfci_data.loc[:, f"pred_{ae}_twosides"] = pd.DataFrame(dfci_test_twosides_scores[700][:, drug_1_ind_inds, drug_2_ind_inds], index=sum([outcome_mapper[ae]["twosides"] for ae in aes], start=[]), columns=list(zip(np.array(drug_names)[drug_1_ind_inds], np.array(drug_names)[drug_2_ind_inds]))).drop_duplicates().loc["Hypercalcaemia"].values

plot_data = dfci_data.query("drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs and num_all_patients >= @detect_thres")
print(kendalltau(plot_data[f"proportion_{ae}"], plot_data[f"pred_{ae}_twosides"]))

SignificanceResult(statistic=0.2905817458203733, pvalue=0.14722696631062424)


### Adjusting for patient characteristics

In [ ]:
include_tissue_type = False  # Because of high collinearity between tumor type and drug combo
top_tissue_only = 10
standardize_scores = True 

In [ ]:
dfci_patient_data_filtered = pd.read_pickle("./dfci/dfci_patient_data_filtered.pkl")
dfci_patient_drugs_twosides_scores = pd.read_pickle(BASE_DIR + "temp/dfci_patient_drugs_twosides_scores.pkl")

assert not dfci_patient_data_filtered.index.has_duplicates
dfci_patient_data_ml = dfci_patient_data_filtered.query("(drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs) and ICD_BASED_TISSUE_TYPE not in @heme_tissues")
dfci_patient_data_ml = dfci_patient_data_ml[~dfci_patient_data_ml["FIRST_DRUG_REGIMEN"].str.contains("PEGYLATEDLIPOSOMALDOXORUBICIN")]  # No match
dfci_patient_data_ml = dfci_patient_data_ml[dfci_patient_data_ml["FIRST_DRUG_REGIMEN"].isin(dfci_patient_data_ml["FIRST_DRUG_REGIMEN"].value_counts()[dfci_patient_data_ml["FIRST_DRUG_REGIMEN"].value_counts() >= detect_thres].index.values)]
dfci_patient_data_ml.loc[dfci_patient_data_ml["RACE"].isin(dfci_patient_data_ml["RACE"].value_counts()[dfci_patient_data_ml["RACE"].value_counts() < 100].index.values), "RACE"] = "OTHER"

# Keep only the 10 most frequent tissue types; everything else --> 'OTHER'
top_tissues = dfci_patient_data_ml["ICD_BASED_TISSUE_TYPE"].value_counts().nlargest(top_tissue_only).index
# Exclude UNSPECIFIED
if "UNSPECIFIED" in top_tissues:
    top_tissues = top_tissues.drop("UNSPECIFIED")
dfci_patient_data_ml["ICD_BASED_TISSUE_TYPE_REDUCED"] = np.where(
    dfci_patient_data_ml["ICD_BASED_TISSUE_TYPE"].isin(top_tissues),
    dfci_patient_data_ml["ICD_BASED_TISSUE_TYPE"],
    "Other",
)

In [76]:
if include_tissue_type:
    ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore", drop=["Other", "OTHER", "MALE"])
    dfci_patient_data_ml_patient_X = pd.DataFrame(ohe.fit_transform(dfci_patient_data_ml[["ICD_BASED_TISSUE_TYPE_REDUCED", "RACE", "GENDER_NM"]]), columns=ohe.get_feature_names_out(), index=dfci_patient_data_ml.index)
else:
    ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore", drop=["OTHER", "MALE"])
    dfci_patient_data_ml_patient_X = pd.DataFrame(ohe.fit_transform(dfci_patient_data_ml[["RACE", "GENDER_NM"]]), columns=ohe.get_feature_names_out(), index=dfci_patient_data_ml.index)
        
dfci_patient_data_ml_patient_X = pd.concat([
    dfci_patient_data_ml[["PALLIATIVE_INTENT", "AGE_AT_TREAT"]], 
    dfci_patient_data_ml_patient_X, 
], axis=1)

In [77]:
aes_score_inds = {
    "neutropenia": 0, 
    "pancytopenia": 0, 
    "anemia": 0,
    "thrombocytopenia": 0, 
    "polyneuropathy": 0,
    "pulmonary_embolism/deep_vein_thrombosis": 1, 
    "acute_kidney_injury": 4, 
    "hyponatremia": 0, 
    "hypokalemia": 0, 
    "hyperkalemia": 0, 
    "hypomagnesemia": 0, 
    "hypocalcemia": 0, 
    "hypercalcemia": 0, 
}

In [ ]:
for ae in aes:
    print("\n"+ae)
    if os.path.exists(f"./dfci/dfci_patient_data_ml_{ae.replace('/', '_')}_coef_table_{detect_thres}_{include_tissue_type}_{top_tissue_only}_{standardize_scores}.csv"):
        print(f"\nSkipping {ae} as it has already been processed.\n")
        continue

    dfci_patient_data_ml_drug_twosides_scores_X = dfci_patient_drugs_twosides_scores.drop_duplicates().T.drop_duplicates().T.loc[[o for o in outcome_mapper[ae]["twosides"] if o not in {"Adverse drug reaction", "Adverse event"}], dfci_patient_data_ml[["drug_name_1", "drug_name_2"]].apply(lambda row: (row["drug_name_1"], row["drug_name_2"]), axis=1).values].T
    dfci_patient_data_ml_drug_twosides_scores_X.columns = [col+"_twosides" for col in dfci_patient_data_ml_drug_twosides_scores_X.columns]
    dfci_patient_data_ml_drug_twosides_scores_X.index = dfci_patient_data_ml_patient_X.index

    # Prepare X and y
    X = [dfci_patient_data_ml_patient_X]
    
    score_ind = aes_score_inds[ae]
    X += [dfci_patient_data_ml_drug_twosides_scores_X.iloc[:, score_ind]]

    X = pd.concat(X, axis=1)
    y = dfci_patient_data_ml[f"AE_{ae}"].values

    def _standardise(df, scaler=None):
        """Z-score numeric (non-binary) cols; keep dummies as-is."""
        df = df.copy()
        num_mask = (df.dtypes == float) | (df.dtypes == int)
        # any numeric col holding only {0,1} is probably a dummy --> skip
        num_cols = [c for c in df.columns[num_mask]
                    if set(df[c].dropna().unique()) - {0, 1}]
        if scaler is None:
            scaler = StandardScaler()
            df[num_cols] = scaler.fit_transform(df[num_cols])
            return df, scaler
        else:
            df[num_cols] = scaler.transform(df[num_cols])
            return df

    # -----------------------------------------------------------
    # 1.  Scale once on the full feature set
    # -----------------------------------------------------------
    if standardize_scores:  # If not float, _standardise will not standardize the scores, which are originally float32 or object
        X = X.astype(float)
        
    y_arr      = y.copy()
    index_arr  = X.index.values  # ndarray of patient IDs
    
    # -----------------------------------------------------------
    # 2.  Hyper-parameter grid
    # -----------------------------------------------------------
    alphas = 10 ** np.arange(-4., 2.5, .5)

    # best_auroc, best_auprc, best_hp = -np.inf, -np.inf, None
    best_auroc, best_hp = -np.inf, None
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for alpha in alphas:
        fold_aurocs = []
        fold_auprcs = []
        for tr_idx, va_idx in skf.split(index_arr, y_arr):
            X_scaled_tr, scaler = _standardise(X.iloc[tr_idx])
            X_np_tr = X_scaled_tr.values.astype(float)
            X_tr = X_np_tr
            X_tr_c = sm.add_constant(X_tr, has_constant='add')
            y_tr = y_arr[tr_idx]

            mod = sm.Logit(y_tr, X_tr_c)
            with warnings.catch_warnings():
                from statsmodels.tools.sm_exceptions import ConvergenceWarning
                warnings.filterwarnings("ignore", category=ConvergenceWarning)
                res = mod.fit_regularized(method="l1", alpha=alpha, maxiter=8_000, cnvrg_tol=1e-8, disp=False)

            # predict on each direction ---------------------
            X_scaled_va = X.iloc[va_idx]
            X_np_va = X_scaled_va.values.astype(float)
            X_va_dup = X_np_va
            X_va_dup_c = sm.add_constant(X_va_dup, has_constant='add')
            agg_pred = res.predict(X_va_dup_c)

            # fold_auprcs.append(average_precision_score(y_arr[va_idx], agg_pred))
            fold_aurocs.append(roc_auc_score(y_arr[va_idx], agg_pred))

        mean_auroc = np.mean(fold_aurocs)
        # mean_auprc = np.mean(fold_auprcs)
        # if (mean_auprc > best_auprc) or ((mean_auprc == best_auprc) and (mean_auroc > best_auroc)):
        if mean_auroc > best_auroc:
            best_auroc, best_hp = mean_auroc, alpha
            # best_auprc = mean_auprc
        # print(f"alpha={alpha:.1e}  -->  AUPRC={mean_auprc:.3f}  AUROC={mean_auroc:.3f}")

    # print(f"\nBEST  AUPRC={best_auprc:.3f}  AUROC={best_auroc:.3f}  at alpha={best_hp:.3g}")
    
    alpha_opt = best_hp
    
    X_scaled, scaler = _standardise(X)
    X_np       = X_scaled.values.astype(float)
    y_arr      = y.copy()
    X_dup = X_np
    y_dup = y_arr
    X_dup_c = sm.add_constant(X_dup, has_constant='add')

    mod_full = sm.Logit(y_dup, X_dup_c)
    with warnings.catch_warnings():
        from statsmodels.tools.sm_exceptions import ConvergenceWarning
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        res_full = mod_full.fit_regularized(method="l1",
                                            cnvrg_tol=1e-8,
                                            alpha=alpha_opt, 
                                            refit=True, 
                                            maxiter=8_000)  # -ll/n + alpha * ((1 - l1_wt) * |param|_2^2/2 + l1_wt * |param|_1)

    # Example: save tidy table of effects
    params   = res_full.params                    # log-odds coefficients
    se       = res_full.bse                       # robust or ML SEs (depends on .fit())
    pvalues  = res_full.pvalues
    ci_lo, ci_hi = res_full.conf_int().T   # log-odds CI limits

    # ---------- build table ----------
    coef_table = (
        pd.DataFrame({
            "feature"   : ["const"] + X_scaled.columns.tolist(),            # includes 'const'
            "beta"      : params,           # log-odds
            "OR"        : np.exp(params),   # odds ratio
            "CI_low"    : np.exp(ci_lo),           # 95 % CI lower bound (OR scale)
            "CI_high"   : np.exp(ci_hi),           # 95 % CI upper bound
            "se"        : se,
            "p"         : pvalues,
        })
        .sort_values("p")                          # smallest p on top
        .reset_index(drop=True)
    )
    
    coef_table.to_csv(f"./dfci/dfci_patient_data_ml_{ae.replace('/', '_')}_coef_table_{detect_thres}_{include_tissue_type}_{top_tissue_only}_{standardize_scores}.csv", index=False)



neutropenia
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.1804871833774824
            Iterations: 152
            Function evaluations: 152
            Gradient evaluations: 152

pancytopenia
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.09009141510045467
            Iterations: 82
            Function evaluations: 82
            Gradient evaluations: 82

anemia
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.2118302964923294
            Iterations: 81
            Function evaluations: 81
            Gradient evaluations: 81

thrombocytopenia
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.12283133429200342
            Iterations: 164
            Function evaluations: 164
            Gradient evaluations: 164

polyneuropathy
Optimization terminated successfully    (Exit mode 0)
            Current function 

### Patient-specific AE prediction

In [ ]:
include_tissue_type = True
top_tissue_only = 10

include_one_hot_drug_pair = False
standardize_scores = False

dfci_patient_data_filtered = pd.read_pickle("./dfci/dfci_patient_data_filtered.pkl")

assert not dfci_patient_data_filtered.index.has_duplicates
dfci_patient_data_ml = dfci_patient_data_filtered.query("(drug_name_1 not in @heme_drugs and drug_name_2 not in @heme_drugs) and ICD_BASED_TISSUE_TYPE not in @heme_tissues")
dfci_patient_data_ml = dfci_patient_data_ml[~dfci_patient_data_ml["FIRST_DRUG_REGIMEN"].str.contains("PEGYLATEDLIPOSOMALDOXORUBICIN")]  # no match
dfci_patient_data_ml.loc[dfci_patient_data_ml["RACE"].isin(dfci_patient_data_ml["RACE"].value_counts()[dfci_patient_data_ml["RACE"].value_counts() < 100].index.values), "RACE"] = "OTHER"

# Keep only the 10 most frequent tissue types; everything else --> 'OTHER'
top5_tissues = dfci_patient_data_ml["ICD_BASED_TISSUE_TYPE"].value_counts().nlargest(top_tissue_only).index
# Exclude UNSPECIFIED
if "UNSPECIFIED" in top5_tissues:
    top5_tissues = top5_tissues.drop("UNSPECIFIED")
dfci_patient_data_ml["ICD_BASED_TISSUE_TYPE_REDUCED"] = np.where(
    dfci_patient_data_ml["ICD_BASED_TISSUE_TYPE"].isin(top5_tissues),
    dfci_patient_data_ml["ICD_BASED_TISSUE_TYPE"],
    "Other",
)

drop_cols = ["MALE"]
patient_data_cols = ["GENDER_NM"]
if include_tissue_type:
    drop_cols += ["Other"]
    patient_data_cols += ["ICD_BASED_TISSUE_TYPE_REDUCED"]
drop_cols += ["OTHER"]
patient_data_cols += ["RACE"]
        
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore", drop=drop_cols)
dfci_patient_data_ml_patient_X = pd.DataFrame(ohe.fit_transform(dfci_patient_data_ml[patient_data_cols]), columns=ohe.get_feature_names_out(), index=dfci_patient_data_ml.index)
dfci_patient_data_ml_patient_X = pd.concat([
    dfci_patient_data_ml[["PALLIATIVE_INTENT", "AGE_AT_TREAT"]],
    dfci_patient_data_ml_patient_X
], axis=1)

print(dfci_patient_data_ml.shape, dfci_patient_data_ml_patient_X.shape)

(3577, 26) (3577, 15)


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

unique_drug_inds = np.unique(dfci_patient_data_ml[["drug_index_1", "drug_index_2"]].values.flatten())
smiles = drug_metadata["canonical_smiles"].loc[unique_drug_inds].values.tolist()

def morgan_fp(smiles: str,
                radius: int = 2,      # ECFP4
                n_bits: int = 32,     # 64-dim output
                use_chirality: bool = True) -> np.ndarray:
    """Return a n_bits-element numpy vector."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(n_bits, dtype=np.uint8)

    bv = AllChem.GetMorganFingerprintAsBitVect(
            mol,
            radius=radius,
            nBits=n_bits,
            useChirality=use_chirality)

    arr = np.zeros(n_bits, dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(bv, arr)
    return arr

drug_fps = {ind: morgan_fp(smi, n_bits=32).astype(float) for ind, smi in zip(unique_drug_inds, smiles)}

In [65]:
dfci_patient_data_ml["FIRST_DRUG_REGIMEN"].nunique()
# print(", ".join(["+".join([d.capitalize() for d in pair.split("+")]) for pair in sorted(dfci_patient_data_ml["FIRST_DRUG_REGIMEN"].unique())]))

26

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier


def _standardise(df, scaler=None):
    """Z-score numeric (non-binary) cols; keep dummies as-is."""
    df = df.copy()
    num_mask = (df.dtypes == float) | (df.dtypes == int)
    # any numeric col holding only {0,1} is probably a dummy --> skip
    num_cols = [c for c in df.columns[num_mask]
                if set(df[c].dropna().unique()) - {0, 1}]
    if scaler is None:
        scaler = StandardScaler()
        df[num_cols] = scaler.fit_transform(df[num_cols])
        return df, scaler
    else:
        df[num_cols] = scaler.transform(df[num_cols])
        return df


# Madrigal
ckpt_aurocs = {}
all_aurocs = {}
all_aurocs_std = {}
for ae in aes:
    for seed in [0,1,2,42,99]:
        for ckpt in twosides_ckpts:
            embeds = torch.load(BASE_DIR + f"model_output/TWOSIDES/split_by_pairs/{ckpt}/all_drug_embeddings_full_700.pt").numpy()
            pca = PCA(n_components=32)
            embeds = pca.fit_transform(embeds)
        
            dfci_patient_data_ml_drug_1_embeds_X = pd.DataFrame(embeds[dfci_patient_data_ml["drug_index_1"].values, :], index=dfci_patient_data_ml_patient_X.index, columns=["madrigal_"+str(i)+"_drug_1" for i in range(embeds.shape[1])])
            dfci_patient_data_ml_drug_2_embeds_X = pd.DataFrame(embeds[dfci_patient_data_ml["drug_index_2"].values, :], index=dfci_patient_data_ml_patient_X.index, columns=["madrigal_"+str(i)+"_drug_2" for i in range(embeds.shape[1])])  # reverse later

            # Prepare X and y
            X = [dfci_patient_data_ml_patient_X]
            X += [dfci_patient_data_ml_drug_1_embeds_X, dfci_patient_data_ml_drug_2_embeds_X]
            X_rev = [dfci_patient_data_ml_patient_X]  # for reverse direction (of a drug pair)
            X_rev += [dfci_patient_data_ml_drug_2_embeds_X.rename(
                columns=dict(zip(dfci_patient_data_ml_drug_2_embeds_X.columns, [col.replace("_drug_2", "_drug_1") for col in dfci_patient_data_ml_drug_2_embeds_X.columns]))
            ), dfci_patient_data_ml_drug_1_embeds_X.rename(
                columns=dict(zip(dfci_patient_data_ml_drug_1_embeds_X.columns, [col.replace("_drug_1", "_drug_2") for col in dfci_patient_data_ml_drug_1_embeds_X.columns]))
            )]
        
            X = pd.concat(X, axis=1)
            X_rev = pd.concat(X_rev, axis=1)
            y = dfci_patient_data_ml[f"AE_{ae}"].values

            if standardize_scores:  # If not float, _standardise will not standardize the scores, which are originally float32 or object
                X = X.astype(float)
                X_rev = X_rev.astype(float)
                
            y_arr = y.copy()
            index_arr = X.index.values
        
            skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

            fold_aurocs = []
            for tr_idx, va_idx in skf.split(index_arr, y_arr):
                X_merged_tr = pd.concat([X.iloc[tr_idx], X_rev.iloc[tr_idx]], axis=0)
                X_merged_scaled_tr, scaler = _standardise(X_merged_tr)  # fit on merged data
                X_scaled_tr = X_merged_scaled_tr.iloc[:len(tr_idx)]  # first half is X
                X_rev_scaled_tr = X_merged_scaled_tr.iloc[len(tr_idx):]  # second half is
                X_np_tr = X_scaled_tr.values.astype(float)
                X_rev_np_tr = X_rev_scaled_tr.values.astype(float)
                X_tr = np.concatenate([X_np_tr, X_rev_np_tr], axis=0)
                y_tr = np.tile(y_arr[tr_idx], 2)

                res = RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=4, random_state=seed, n_jobs=-1)
                with warnings.catch_warnings():
                    from sklearn.exceptions import ConvergenceWarning
                    warnings.filterwarnings("ignore", category=ConvergenceWarning)
                    res.fit(X_tr, y_tr)
            
                X_merged_va = pd.concat([X.iloc[va_idx], X_rev.iloc[va_idx]], axis=0)
                X_merged_scaled_va = _standardise(X_merged_va, scaler)  # fit on merged data
                X_scaled_va = X_merged_scaled_va.iloc[:len(va_idx)]  # first half is X
                X_rev_scaled_va = X_merged_scaled_va.iloc[len(va_idx):]
                X_np_va = X_scaled_va.values.astype(float)
                X_rev_np_va = X_rev_scaled_va.values.astype(float)
                X_va_dup = np.concatenate([X_np_va, X_rev_np_va], axis=0)
                
                agg_pred = res.predict_proba(X_va_dup)[:, 1]
                agg_pred = agg_pred.reshape(2, -1).mean(0)

                fold_aurocs.append(roc_auc_score(y_arr[va_idx], agg_pred))

            mean_auroc = np.mean(fold_aurocs)
            ckpt_aurocs.setdefault(ae, []).append(mean_auroc)
        
    all_aurocs[ae] = np.mean(ckpt_aurocs[ae])
    all_aurocs_std[ae] = np.std(ckpt_aurocs[ae])


# BASELINE: Patient characteristics + Morgan FP
all_aurocs_baseline = {}
all_aurocs_baseline_std = {}
dfci_patient_data_ml_patient_X_baseline = dfci_patient_data_ml_patient_X.copy()
for ae in aes:
    for seed in [0,1,2,42,99]:
        dfci_patient_data_ml_drug_1_fps_X = pd.DataFrame(
            np.stack(dfci_patient_data_ml["drug_index_1"].apply(lambda x: drug_fps[x]).values, axis=0),
            columns=["morgan_"+str(i)+"_drug_1" for i in range(list(drug_fps.values())[0].shape[0])],
            index=dfci_patient_data_ml.index
        )
        dfci_patient_data_ml_drug_2_fps_X = pd.DataFrame(
            np.stack(dfci_patient_data_ml["drug_index_2"].apply(lambda x: drug_fps[x]).values, axis=0),
            columns=["morgan_"+str(i)+"_drug_2" for i in range(list(drug_fps.values())[0].shape[0])],
            index=dfci_patient_data_ml.index
        )
            
        # Prepare X and y
        X = [dfci_patient_data_ml_patient_X_baseline]
        X += [dfci_patient_data_ml_drug_1_fps_X, dfci_patient_data_ml_drug_2_fps_X]
        X_rev = [dfci_patient_data_ml_patient_X_baseline]  # for reverse direction (of a drug pair)
        X_rev += [dfci_patient_data_ml_drug_2_fps_X.rename(
            columns=dict(zip(dfci_patient_data_ml_drug_2_fps_X.columns, [col.replace("_drug_2", "_drug_1") for col in dfci_patient_data_ml_drug_2_fps_X.columns]))
        ), dfci_patient_data_ml_drug_1_fps_X.rename(
            columns=dict(zip(dfci_patient_data_ml_drug_1_fps_X.columns, [col.replace("_drug_1", "_drug_2") for col in dfci_patient_data_ml_drug_1_fps_X.columns]))
        )]
        
        X = pd.concat(X, axis=1)
        X_rev = pd.concat(X_rev, axis=1)
        y = dfci_patient_data_ml[f"AE_{ae}"].values
        
        if standardize_scores:  # If not float, _standardise will not standardize the scores, which are originally float32 or object
            X = X.astype(float)
            X_rev = X_rev.astype(float)
                
        y_arr      = y.copy()
        index_arr  = X.index.values  # ndarray of patient IDs
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        fold_aurocs = []
        for tr_idx, va_idx in skf.split(index_arr, y_arr):
            X_merged_tr = pd.concat([X.iloc[tr_idx], X_rev.iloc[tr_idx]], axis=0)
            X_merged_scaled_tr, scaler = _standardise(X_merged_tr)  # fit on merged data
            X_scaled_tr = X_merged_scaled_tr.iloc[:len(tr_idx)]  # first half is X
            X_rev_scaled_tr = X_merged_scaled_tr.iloc[len(tr_idx):]  # second half is
            X_np_tr = X_scaled_tr.values.astype(float)
            X_rev_np_tr = X_rev_scaled_tr.values.astype(float)
            X_tr = np.concatenate([X_np_tr, X_rev_np_tr], axis=0)
            y_tr = np.tile(y_arr[tr_idx], 2)

            res = RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=4, random_state=seed, n_jobs=-1)
            with warnings.catch_warnings():
                from sklearn.exceptions import ConvergenceWarning
                warnings.filterwarnings("ignore", category=ConvergenceWarning)
                res.fit(X_tr, y_tr)
        
            X_merged_va = pd.concat([X.iloc[va_idx], X_rev.iloc[va_idx]], axis=0)
            X_merged_scaled_va = _standardise(X_merged_va, scaler)  # fit on merged data
            X_scaled_va = X_merged_scaled_va.iloc[:len(va_idx)]  # first half is X
            X_rev_scaled_va = X_merged_scaled_va.iloc[len(va_idx):]
            X_np_va = X_scaled_va.values.astype(float)
            X_rev_np_va = X_rev_scaled_va.values.astype(float)
            X_va_dup = np.concatenate([X_np_va, X_rev_np_va], axis=0)
            
            agg_pred = res.predict_proba(X_va_dup)[:, 1]
            agg_pred = agg_pred.reshape(2, -1).mean(0)

            fold_aurocs.append(roc_auc_score(y_arr[va_idx], agg_pred))
            
    all_aurocs_baseline[ae] = np.mean(fold_aurocs)
    all_aurocs_baseline_std[ae] = np.std(fold_aurocs)


# BASELINE: One-hot regimen encoding
drop_cols = ["MALE"]
patient_data_cols = ["GENDER_NM"]
if include_tissue_type:
    drop_cols += ["Other"]
    patient_data_cols += ["ICD_BASED_TISSUE_TYPE_REDUCED"]
drop_cols += ["OTHER"]
patient_data_cols += ["RACE"]
drop_cols += ["CARBOPLATIN+PACLITAXEL"]
patient_data_cols += ["FIRST_DRUG_REGIMEN"]
        
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore", drop=drop_cols)
dfci_patient_data_ml_patient_X_baseline = pd.DataFrame(ohe.fit_transform(dfci_patient_data_ml[patient_data_cols]), columns=ohe.get_feature_names_out(), index=dfci_patient_data_ml.index)
dfci_patient_data_ml_patient_X_baseline = pd.concat([
    dfci_patient_data_ml[["PALLIATIVE_INTENT", "AGE_AT_TREAT"]],
    dfci_patient_data_ml_patient_X_baseline
], axis=1)

all_aurocs_ohe = {}
all_aurocs_ohe_std = {}
for ae in aes:
    for seed in [0,1,2,42,99]:
        # Prepare X and y
        X = dfci_patient_data_ml_patient_X_baseline.copy()
        y = dfci_patient_data_ml[f"AE_{ae}"].values
        
        if standardize_scores:  # If not float, _standardise will not standardize the scores, which are originally float32 or object
            X = X.astype(float)
            
        y_arr      = y.copy()
        index_arr  = X.index.values  # ndarray of patient IDs
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        fold_aurocs = []
        for tr_idx, va_idx in skf.split(index_arr, y_arr):
            X_merged_tr = X.iloc[tr_idx]
            X_scaled_tr, scaler = _standardise(X_merged_tr)  # fit on merged data
            X_np_tr = X_scaled_tr.values.astype(float)

            X_tr = X_np_tr
            y_tr = y_arr[tr_idx]

            res = RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=4, random_state=seed, n_jobs=-1)
            with warnings.catch_warnings():
                from sklearn.exceptions import ConvergenceWarning
                warnings.filterwarnings("ignore", category=ConvergenceWarning)
                res.fit(X_tr, y_tr)
        
            X_merged_va = X.iloc[va_idx]
            X_scaled_va = _standardise(X_merged_va, scaler)  # fit on merged data
            X_np_va = X_scaled_va.values.astype(float)
            
            X_va_dup = X_np_va
            agg_pred = res.predict_proba(X_va_dup)[:, 1]

            fold_aurocs.append(roc_auc_score(y_arr[va_idx], agg_pred))

    all_aurocs_ohe[ae] = np.mean(fold_aurocs)
    all_aurocs_ohe_std[ae] = np.std(fold_aurocs)
    

df = pd.DataFrame.from_dict({
    "madrigal_aurocs": all_aurocs,
    "madrigal_aurocs_std": all_aurocs_std,
    "morgan_fps_aurocs": all_aurocs_baseline,
    "morgan_fps_aurocs_std": all_aurocs_baseline_std,
    "one_hot_regimen_aurocs": all_aurocs_ohe,
    "one_hot_regimen_aurocs_std": all_aurocs_ohe_std,
}, orient="columns")

In [67]:
df

,madrigal_aurocs,madrigal_aurocs_std,morgan_fps_aurocs,morgan_fps_aurocs_std,one_hot_regimen_aurocs,one_hot_regimen_aurocs_std
neutropenia,0.656848,0.003116,0.628628,0.043011,0.610829,0.043159
pancytopenia,0.742523,0.005900,0.728668,0.054139,0.734023,0.051141
anemia,0.645019,0.003821,0.641327,0.034961,0.638308,0.022005
thrombocytopenia,0.752950,0.003215,0.748996,0.058050,0.739952,0.062810
polyneuropathy,0.706855,0.005559,0.696617,0.073744,0.693905,0.081625
pulmonary_embolism/deep_vein_thrombosis,0.675816,0.004416,0.700503,0.022182,0.689393,0.017375
acute_kidney_injury,0.681986,0.004559,0.688367,0.090336,0.661179,0.089250
hyponatremia,0.681373,0.002027,0.679357,0.036780,0.681781,0.043588
hypokalemia,0.572145,0.004039,0.561256,0.036973,0.568751,0.036852
hyperkalemia,0.712507,0.009736,0.660941,0.133960,0.657905,0.146322
